In [1]:
# Homework8 Solution

import requests
import bs4
import pandas
import numpy

In [2]:
# Problem 1:
page = requests.get('http://www.ncaa.com/rankings/basketball-men/d1/ncaa-mens-basketball-rpi')
soup = bs4.BeautifulSoup(page.content, 'html.parser')
tb = soup.find('table', {'class':'ncaa-rankings-table'}).find('tbody')
rows = tb.find_all('tr')
data = [(r.contents[3].text, int(r.contents[0].text), r.contents[2].text) \
       for r in rows]
table = pandas.DataFrame(data, columns=['Conference', 'Rank', 'Team'])
table

,Conference,Rank,Team
0,Atlantic Coast,1,North Carolina
1,Southeastern,2,Kentucky
2,Big East,3,Villanova
3,Big 12,4,Kansas
4,Pac-12,5,Arizona
5,West Coast,6,Gonzaga
6,Pac-12,7,Oregon
7,Southeastern,8,Florida
8,Big 12,9,Baylor
9,Atlantic Coast,10,Duke


In [3]:
g = table.groupby('Conference')
ranks = g.Rank.mean().sort_values()
ranks

Conference
Atlantic Coast       59.133333
Big 12               64.000000
Big Ten              66.857143
Big East             74.800000
Southeastern         84.785714
Pac-12              101.666667
Mountain West       129.636364
Atlantic 10         131.785714
AAC                 135.454545
West Coast          141.900000
Colonial            160.800000
Sun Belt            172.666667
Missouri Valley     172.900000
Mid-American        179.083333
Metro Atlantic      181.090909
Southern            182.200000
Horizon             185.500000
Western Athletic    188.125000
Summit League       192.777778
Ivy League          205.625000
America East        214.333333
Conference USA      214.428571
Patriot             215.300000
Big South           217.900000
OVC                 219.166667
ASUN                239.500000
Southland           240.384615
Big Sky             250.750000
Big West            259.777778
Northeast           262.900000
Southwestern        296.900000
Mid-Eastern         306.6153

In [4]:
# Problem 2:
teams = [(t.text, t.attrs['href']) for t in 
    bs4.BeautifulSoup(requests.get('http://www.ncaa.com/standings/basketball-men/d1/2016/atlantic-coast').content, 'html.parser')  
    .find('div', {'class':'ncaa-standings-conference-show-atlantic-coast'}) 
    .find_all('a', {'class':'ncaa-standing-conference-team-link'}) ]
teams

[('North Carolina', '/schools/north-carolina/basketball-men'),
 ('Florida State', '/schools/florida-st/basketball-men'),
 ('Louisville', '/schools/louisville/basketball-men'),
 ('Notre Dame', '/schools/notre-dame/basketball-men'),
 ('Duke', '/schools/duke/basketball-men'),
 ('Virginia', '/schools/virginia/basketball-men'),
 ('Virginia Tech', '/schools/virginia-tech/basketball-men'),
 ('Miami (FL)', '/schools/miami-fl/basketball-men'),
 ('Syracuse', '/schools/syracuse/basketball-men'),
 ('Wake Forest', '/schools/wake-forest/basketball-men'),
 ('Georgia Tech', '/schools/georgia-tech/basketball-men'),
 ('Clemson', '/schools/clemson/basketball-men'),
 ('Pittsburgh', '/schools/pittsburgh/basketball-men'),
 ('North Carolina State', '/schools/north-carolina-st/basketball-men'),
 ('Boston College', '/schools/boston-college/basketball-men')]

In [5]:
margins = [(team, numpy.mean(
    [eval(score[2:]) * (1 if score[0]=='W' else -1) for score in 
        [game.contents[2].text for game in 
            bs4.BeautifulSoup(requests.get('http://www.ncaa.com' + url).content,'html.parser') 
                .find('table', {'class':'ncaa-schools-sport-table'}).tbody 
                .find_all('tr') ] ]) )
    for (team,url) in teams]
margins

[('North Carolina', 13.824999999999999),
 ('Florida State', 10.0),
 ('Louisville', 11.323529411764707),
 ('Notre Dame', 8.0277777777777786),
 ('Duke', 10.648648648648649),
 ('Virginia', 9.7058823529411757),
 ('Virginia Tech', 4.3939393939393936),
 ('Miami (FL)', 4.9090909090909092),
 ('Syracuse', 5.2352941176470589),
 ('Wake Forest', 4.3939393939393936),
 ('Georgia Tech', 0.64864864864864868),
 ('Clemson', 3.2424242424242422),
 ('Pittsburgh', -1.8181818181818181),
 ('North Carolina State', -2.0),
 ('Boston College', -5.875)]

In [6]:
table = pandas.DataFrame(margins, columns=['Team', 'Margin']) \
    .sort_values('Margin', ascending=False)
table

,Team,Margin
0,North Carolina,13.825000
2,Louisville,11.323529
4,Duke,10.648649
1,Florida State,10.000000
5,Virginia,9.705882
3,Notre Dame,8.027778
8,Syracuse,5.235294
7,Miami (FL),4.909091
6,Virginia Tech,4.393939
9,Wake Forest,4.393939
